In [ ]:
# Step 1: 必要なライブラリをインポート
import re
import urllib.request

# Step 2: 青空文庫の「こころ」テキストデータ（ルビあり）をダウンロード
url = "https://www.aozora.gr.jp/cards/000148/files/773_14560.html"
file_name = "kokoro_raw.html"
urllib.request.urlretrieve(url, file_name)

# Step 3: HTMLから本文テキストだけを取り出す
from bs4 import BeautifulSoup

with open(file_name, encoding='shift_jis') as f:
  soup = BeautifulSoup(f, "html.parser")
  # テキスト抽出 (<div class="main_text>") から
  text = soup.find("div", class_="main_text").get_text()

# Step 4: ルビ削除関数
def remove_ruby(text):
  text = re.sub(r'｜([^《]+)《[^》]+》', r'\1', text)
  text = re.sub(r'《[^》]+》', '', text)
  return text

# Step 5: ヘッダー・フッター・空行を削除
def clean_text(text):
  text = remove_ruby(text)
  # 不要な空行や記号を消す
  text = re.sub(r'\n\s*\n', '\n', text)
  text = re.sub(r'［＃.*?］', '', text)  # 青空注記
  return text.strip()

# Step 6: 実行して整形
cleaned = clean_text(text)

# Step 7: 保存(GPT学習用にテキストファイル化)
with open("kokoro_clean.txt", "w", encoding="utf-8") as f:
  f.write(cleaned)

print("整形完了！ファイル名： kokoro_clean.txt")

整形完了！ファイル名： kokoro_clean.txt


In [61]:
# 夏目漱石「坊ちゃん」をダウンロード
url = "https://www.aozora.gr.jp/cards/000148/files/752_14964.html"
file_name = "botchan_raw.html"
urllib.request.urlretrieve(url, file_name)

# HTMLから本文を取り出す
with open(file_name, encoding='shift_jis') as f:
    soup = BeautifulSoup(f, "html.parser")
    text = soup.find("div", class_="main_text").get_text()

# テキスト整形関数を再利用
cleaned_botchan = clean_text(text)

# 保存
with open("botchan_clean.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_botchan)

print("坊ちゃん整形完了！")


坊ちゃん整形完了！


In [98]:
# 夏目漱石「正岡子規」をダウンロード
url = "https://www.aozora.gr.jp/cards/000148/files/1751_6496.html"
file_name = "masaoka_raw.html"
urllib.request.urlretrieve(url, file_name)

# HTMLから本文を取り出す
with open(file_name, encoding='shift_jis') as f:
    soup = BeautifulSoup(f, "html.parser")
    text = soup.find("div", class_="main_text").get_text()

# テキスト整形関数を再利用
cleaned_masaoka = clean_text(text)

# 保存
with open("masaoka_clean.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_botchan)

print("正岡子規整形完了！")


正岡子規整形完了！


In [99]:
# こころと、坊ちゃん、正岡子規のデータを結合
with open("kokoro_clean.txt", "r", encoding="utf-8") as f:
  kokoro_text = f.read()

with open("botchan_clean.txt", "r", encoding="utf-8") as f:
  botchan_text = f.read()

with open("masaoka_clean.txt", "r", encoding="utf-8") as f:
  masaoka_text = f.read()

# 結合
combined_text = kokoro_text + "\n" + botchan_text + "\n" + masaoka_text

# 保存
with open("combined_clean.txt", "w", encoding="utf-8") as f:
  f.write(combined_text)

print("データ結合完了!")

データ結合完了!


In [100]:
# ファイル読み込み
with open("combined_clean.txt", encoding="utf-8") as f:
  text = f.read()

# ユニークな文字を取り出す
chars = sorted(list(set(text)))

# 空白文字を追加
chars.append(' ')
vocab_size = len(chars)

# 文字 ⇔ ID の辞書を作る
stoi = {ch: i for i, ch in enumerate(chars)}  # string to int
itos = {i: ch for ch, i in stoi.items()}       # int to string

# エンコード・デコード関数
def encode(s):
  return [stoi[c] for c in s]

def decode(ids):
  return ''.join([itos[i] for i in ids])

print(f"語彙サイズ（ユニークな文字数）: {vocab_size}")
print(f"例:「私」→ {encode('私')}, {decode(encode('私'))}")

語彙サイズ（ユニークな文字数）: 2465
例:「私」→ [1623], 私


In [101]:
# バッチ作成用のコード
import torch

# データ全体の数値化（前処理済み）
data = torch.tensor(encode(text), dtype=torch.long)

# 訓練用と検証用に分ける（9:1くらい）
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# シーケンス長（どれくらいの長さで学習するか）
block_size = 8

# バッチ作成関数
def get_batch(split, batch_size=4):
  data_split = train_data if split == 'train' else val_data
  ix = torch.randint(len(data_split) - block_size, (batch_size,))

  x = torch.stack([data_split[i:i+block_size] for i in ix])
  y = torch.stack([data_split[i+1:i+block_size+1] for i in ix])

  return x, y

In [102]:
# バッチ作成
data = torch.tensor(encode(text), dtype=torch.long)

# 訓練用と検証用に分ける（9:1くらい）
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# バッチ作成関数
def get_batch(split, batch_size=4):
    data_split = train_data if split == 'train' else val_data
    ix = torch.randint(len(data_split) - block_size, (batch_size,))

    x = torch.stack([data_split[i:i+block_size] for i in ix])
    y = torch.stack([data_split[i+1:i+block_size+1] for i in ix])

    return x, y

In [103]:
# テスト
x, y = get_batch('train')

for i in range(4):
  print("入力: ", decode(x[i].tolist()))
  print("正解: ", decode(y[i].tolist()))
  print()

入力:  た説諭を加えた。
正解:  説諭を加えた。新

入力:  一人一人の前へ行
正解:  人一人の前へ行っ

入力:  やすい所を空けて
正解:  すい所を空けて、

入力:  さんに伺っていい
正解:  んに伺っていい質



In [104]:
# 1. モデルの作成
import torch
import torch.nn as nn

# モデルクラス
class CharRNN(nn.Module):
  def __init__(self, vocab_size, embed_size, hidden_size, block_size):
    super(CharRNN, self).__init__()
    self.block_size = block_size
    self.embedding = nn.Embedding(vocab_size, embed_size)
    self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
    self.fc = nn.Linear(hidden_size, vocab_size)

  def forward(self, x, h0=None):
    # 埋め込み層
    x = self.embedding(x)
    # RNN層
    out, h0 = self.rnn(x, h0)
    # 最終層の出力
    out = self.fc(out)
    return out, h0



In [120]:
# 2. モデルの初期化
# ハイパーパラメータ
vocab_size = len(chars)  # 使用する文字の数
embed_size = 64  # 埋め込み層のサイズ
hidden_size = 512  # 隠れ層のサイズ
block_size = 8  # 入力サイズ

# モデルの初期化
model = CharRNN(vocab_size, embed_size, hidden_size, block_size)

# 損失関数 (CrossEntropyLoss)
loss_fn = nn.CrossEntropyLoss()

# 最適化関数 (Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [121]:
# 3. 学習の実行
epochs = 15000
for epoch in range(epochs):
  model.train()
  optimizer.zero_grad()  # 勾配の初期化

  # バッチデータを取得
  x_batch, y_batch = get_batch('train')  # 'train'を使って訓練データを取得

  # モデルの予測
  output, _ = model(x_batch)

  # 損失計算
  loss = loss_fn(output.view(-1, vocab_size), y_batch.view(-1))

  # 誤差逆伝播
  loss.backward()

  # パラメータの更新
  optimizer.step()

  if epoch % 100 == 0:  # 100回ごとに損失を表示
    print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 7.815550327301025
Epoch 100, Loss: 4.987680912017822
Epoch 200, Loss: 5.18065881729126
Epoch 300, Loss: 4.88184928894043
Epoch 400, Loss: 3.872058153152466
Epoch 500, Loss: 4.3108229637146
Epoch 600, Loss: 4.652576446533203
Epoch 700, Loss: 4.68040132522583
Epoch 800, Loss: 3.9063515663146973
Epoch 900, Loss: 4.116420269012451
Epoch 1000, Loss: 5.202846050262451
Epoch 1100, Loss: 4.160465240478516
Epoch 1200, Loss: 4.524961948394775
Epoch 1300, Loss: 3.7046256065368652
Epoch 1400, Loss: 4.0689778327941895
Epoch 1500, Loss: 4.217362880706787
Epoch 1600, Loss: 3.958566427230835
Epoch 1700, Loss: 3.5431199073791504
Epoch 1800, Loss: 3.9945807456970215
Epoch 1900, Loss: 4.518771648406982
Epoch 2000, Loss: 3.9191927909851074
Epoch 2100, Loss: 3.3753879070281982
Epoch 2200, Loss: 2.48144793510437
Epoch 2300, Loss: 4.096367359161377
Epoch 2400, Loss: 3.4752001762390137
Epoch 2500, Loss: 4.134529113769531
Epoch 2600, Loss: 6.295478820800781
Epoch 2700, Loss: 4.041027545928955
Ep

In [122]:
# 4. モデルによる文字生成
# モデルの評価モードに切り替え
model.eval()

# 初期の文字列を設定
start_str = "吾輩は"

# 文字列を生成する関数
def generate(model, start_str, max_len=100, temperature=1.0):
  # 文字列のリストをインデックスに変換
  input_str = [stoi.get(c, stoi[' ']) for c in start_str]
  # input_tensor = torch.tensor(input_str).unsqueeze(0)  # バッチサイズ１で入力
  input_tensor = torch.tensor(input_str).unsqueeze(0)

  generated_str = start_str  # 初期文字列を保存

  # 初期隠れ状態を設定
  h0 = None

  for _ in range(max_len):
    # モデルの出力を取得
    output, h0 = model(input_tensor, h0)

    # 出力の最後の時刻を取り出し、確率分布を計算
    logits = output[0, -1, :] / temperature
    probabilities = torch.softmax(logits, dim=-1)

    # 次の文字を確率的にサンプリング
    next_char_idx = torch.multinomial(probabilities, 1).item()

    # インデックスを文字に戻す
    # next_char = stoi[next_char_idx]
    next_char = itos.get(next_char_idx, ' ')

    # 生成された文字を追加
    generated_str += next_char

    # 次の入力として最後に生成した文字を使用
    input_tensor = torch.tensor([next_char_idx]).unsqueeze(0)

  return generated_str


In [123]:
# 初期の文字列を設定
start_str = "吾輩は"

# 文字列を生成
generated_text = generate(model, start_str, max_len=100, temperature=0.7)

# 生成された文字列を表示
print("生成された文字列:")
print(generated_text)

生成された文字列:
吾輩は温泉はないように黒い出の間の前がありました。その自分の美しかりまではかっただけれ、あとんから読まらんじゃない。二三度
「私の前に立くないで、どった。先生は、私の運命を吹き消えないのです。Ｋが父といった
